# Part 1: Extracting Chemical Named Entities

## 1. Obtaining Data

**ChemDNER Corpus v1.0**

The ChemDNER corpus consists of 10,000 PubMED abstracts and their corresponding label sets of named chemical entities. This data set is [publicly available](http://www.biocreative.org/resources/biocreative-iv/chemdner-corpus/) and can be downloaded directly using the shell script 

    load_data.sh

## 2. Extracting Candidate Mentions
For the extraction step, our goal is to get the highest recall as possible. In cases where we have labeled data, it's easy to get a recall estimate for our extraction pipline.

In [2]:
import umls
import operator
from lexicons.matchers import * 
from utils import unescape_penn_treebank
from ddlite import *
from datasets import *

In [3]:
# parse our corpus, saving CoreNLP results at cache_path
parser = SentenceParser()
corpus = ChemdnerCorpus('datasets/chemdner_corpus/', parser=parser, cache_path="/users/fries/desktop/cache/")

# load the first 100 training documents and collapse all sentences into a single list
pmids = [pmid for pmid in corpus.cv["training"].keys()[:100]]
documents = {pmid:corpus[pmid]["sentences"] for pmid in pmids}
sentences = reduce(lambda x,y:x+y, documents.values())

# load gold annotation tags
annotations = [corpus.annotations[pmid] for pmid in pmids if pmid in corpus.annotations]
annotations = reduce(lambda x,y:x+y, annotations)
annotations = [a.text for a in annotations]

print("%d PubMed abstracts" % len(documents))
print("%d true chemical entity mentions" % len(annotations))
word_n = sum([len(sent.words) for sent in sentences])
print("%d tokens" % word_n)

100 PubMed abstracts
894 true chemical entity mentions
24147 tokens


### Using Matchers

The easiest way to identify candidates is through simple string matching using a dictionary of known entity names. Curating good lexicons can take some time, so we use pre-existing dictionaries provided by the *tmChem* tagger and a UMLS dictionary of all *Substance* semantic types (see the UMLS notebook for instructions how to create arbitrary dictionaries).

In [7]:
regex_fnames = ["datasets/regex/chemdner/patterns.txt"]

# dictionaries from tmChem & the UMLS
dict_fnames = ["datasets/dictionaries/chemdner/mention_chemical.txt",
              "datasets/dictionaries/chemdner/chebi.txt",
              "datasets/dictionaries/chemdner/addition.txt",
              "datasets/dictionaries/umls/substance-sab-all.txt"]

chemicals = []
for fname in dict_fnames:
    chemicals += [line.strip().split("\t")[0] for line in open(fname,"rU").readlines()]
    
regexes = []
for fname in regex_fnames:
    regexes += [line.strip() for line in open(fname,"rU").readlines()]   

# create matchers and extract candidates
extr1 = DictionaryMatch('C', chemicals, ignore_case=True)
#extr2 = AllUpperNounsMatcher('C')
#extr3 = RegexMatch('C', regexes[0], ignore_case=True)
#extr4 = RegexMatch('C', regexes[1], ignore_case=False)
#extr5 = RegexMatch('C', regexes[2], ignore_case=False)
matcher = MultiMatcher(extr1)#, extr3, extr4, extr5)

candidates = Entities(sentences, matcher)

# Crude recall estimate (ignores actual span match and tokenization problems)
mentions = [" ".join(unescape_penn_treebank([e.words[i] for i in e.idxs])) for e in candidates]
gold_mentions = [term for term in annotations]

for m in mentions:
    if m in gold_mentions:
        gold_mentions.remove(m)
tp = len(annotations) - len(gold_mentions)

print("Found %d candidate entities" % len(candidates))
print("Candidates: %.2f%% of all tokens" % (len(candidates)/float(word_n) * 100) )
print("Annotations %.2f%% of all tokens" % (len(annotations)/float(word_n) * 100) )

print("~recall: %.2f (%d/%d)" % (float(tp) / len(annotations), tp, len(annotations)))

Found 5077 candidate entities
Candidates: 21.03% of all tokens
Annotations 3.70% of all tokens
~recall: 0.62 (555/894)


### Error Analysis
For distant supervision, we want our candidate set to have high recall. We currently fall short for chemical named entities. If we look at the gold standard annotations, we can see why our string matching misses some entities. Note how paranthesis and other tokenization issues result in many missed entities. 

In [8]:
mentions = {term:1 for term in mentions}
missed = [term for term in annotations if term not in mentions]
missed = {term:missed.count(term) for term in missed}

for term in sorted(missed.items(),key=operator.itemgetter(1),reverse=1):
    print("%s: %d" % (term[0], missed[term[0]]))

BPA: 16
ZnS: 11
Res: 9
cannabisin B: 7
ABG: 7
DINCH: 7
(13)C: 6
DC-Ch: 6
DOTAP: 5
AcOEt: 5
MINCH: 5
DCA-Tam: 4
hydroxy: 4
organochlorine: 4
LA-Tam: 4
Li(3)PS(4): 3
CA-Tam: 3
TamNHMe: 3
H(2)S: 3
OLL: 3
N2: 3
HEX: 3
dC5'p: 2
CdSe: 2
aflatoxin B(1): 2
dA5'p: 2
T5'p: 2
(1)H: 2
l-amino acid: 2
Lys: 2
5-Aza-dC: 2
monoester: 2
CHDA: 2
HPβCD: 2
dimeric phloroglucinols: 2
3β-{N-[(N',N'-dimethylamino)ethyl]carbamoyl}cholesterol: 2
DMBPA: 2
dG5'p: 2
BPF: 2
diisononyl-cyclohexane-1,2-dicarboxylate: 2
BPZ: 2
Cannabisin B: 2
Rha: 2
CdS: 2
OOL: 2
tetralone: 2
NAD: 1
dicarbonyls: 1
Z-Ligustilide: 1
gallates: 1
6,8-dihydroxy-3-methyl-3,4-dihydroisocoumarin: 1
2'-deoxyadenosine-5'-monophosphate: 1
SP-600125: 1
4-methyl: 1
E-H: 1
Iron aminoimides: 1
1-oleoyl-2-linoleoyl-3-linolenoyl: 1
γ-Aminobutyric acid: 1
2'-deoxycytidine-5'-monophosphate: 1
6-carboxy: 1
dioleoyl linoleoyl: 1
H-H: 1
C2H3OH: 1
(31)P: 1
2,7,12-tris(α-pyridinio-p-tolyl)-17-(p-(methoxymethyl)phenyl) porphycene: 1
haem: 1
Axistatins 1-3: 1

In [6]:
# dump candidates to a pickle
candidates.dump_candidates("/tmp/chemdner_candidate_mentions.pkl")

### Example Best-in-class Tagger

The winning system in the 2013 BioCreative IV CHEMDNER task was tmChem which used 2 linear chain conditional random fields (CRF) with different tokenziation approaches and feature sets.

| Model       | Precision | Recall | F1     |
|-------------|-----------|--------|--------|
| Model 1     | 0.8595    | 0.8721 | 0.8657 |
| Model 2     | **0.8909**    | 0.8575 | **0.8739** |
| Heuristic Combination     | 0.8516    | 0.8906 | 0.8706 |
| Highest Recall | 0.7672    | **0.9212** | 0.8372 |

Leaman, Robert, Chih-Hsuan Wei, and Zhiyong Lu. ["tmChem: a high performance approach for chemical named entity recognition and normalization."](http://www.ncbi.nlm.nih.gov/pmc/articles/PMC4331693/) J. Cheminformatics 7.S-1 (2015): S3.